# Sudoku Board coding

### Author: Umberto Michelucci (C) 2017 

This notebook contains code that can solve any Sudoku with a unique solution in Python. It is written in Python 3 and don't contain any special packages, so you should be able to run it with a pretty standard Python installation (Anaconda for example).

We first start by creating vectors and matrices that contains all the cell names. We will code the name giving to each row a letter and to each column a number.

In [6]:
rows = 'ABCDEFGHI'
cols = '123456789'

We need a helper function, `cross(a, b)`, which, given two strings — a and b — will return the list formed by all the possible concatenations of a letter s in string a with a letter t in string b. In this way we can get each box separately.

In [7]:
def cross(a, b):
      return [s+t for s in a for t in b]

Let's test it with our rows and cols. Each position in our Sudoku board is called a box.
Our cross() function will return all the boxes moving from the top to the right, giving the first row, and then passing to the subsequents rows. So it will start with A1 until A9, then go to the second row and give B1 until B9 and so on.

In [12]:
boxes = cross(rows,cols)
boxes[0:10] # First 10 boxes

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1']

Now let's find the rows, cols and units

In [14]:
row_units = [cross(r, cols) for r in rows]
# For example:
# row_units[0] = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']

column_units = [cross(rows, c) for c in cols]
# For example:
# column_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']

square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
# For example:
# square_units[0] = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']

unitlist = row_units + column_units + square_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

Note that with peers of a specific box, we intend all the units that are in a unit (the 3x3 box that contains the box), in the column of the box and in the row of the box.

Let's check the peers of box "A1". Remember `peers` is a dictionary.

In [18]:
peers["A1"]

{'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'B1',
 'B2',
 'B3',
 'C1',
 'C2',
 'C3',
 'D1',
 'E1',
 'F1',
 'G1',
 'H1',
 'I1'}

In [37]:
square_units

[['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'],
 ['A4', 'A5', 'A6', 'B4', 'B5', 'B6', 'C4', 'C5', 'C6'],
 ['A7', 'A8', 'A9', 'B7', 'B8', 'B9', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'E1', 'E2', 'E3', 'F1', 'F2', 'F3'],
 ['D4', 'D5', 'D6', 'E4', 'E5', 'E6', 'F4', 'F5', 'F6'],
 ['D7', 'D8', 'D9', 'E7', 'E8', 'E9', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3'],
 ['G4', 'G5', 'G6', 'H4', 'H5', 'H6', 'I4', 'I5', 'I6'],
 ['G7', 'G8', 'G9', 'H7', 'H8', 'H9', 'I7', 'I8', 'I9']]

# grid_values()

Now let's implement the function that will give a dictionary with a digit and its coordinates

## TODO: rewrite the following function

In [23]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)


In [205]:
#from utils import *

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = list(grid)
    values = [x if x != '.' else '123456789' for x in values]
    return (dict(zip(boxes, values)))

def grid_values_empty(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    values = list(grid)
    values = [x if x != '.' else '.' for x in values]
    return (dict(zip(boxes, values)))

### Test Cell...

In [29]:
tst = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

values = list(tst)

values = [x if x != '.' else '123456789' for x in values]
print(values)
    
dictionary = dict(zip(boxes, values))

['123456789', '123456789', '3', '123456789', '2', '123456789', '6', '123456789', '123456789', '9', '123456789', '123456789', '3', '123456789', '5', '123456789', '123456789', '1', '123456789', '123456789', '1', '8', '123456789', '6', '4', '123456789', '123456789', '123456789', '123456789', '8', '1', '123456789', '2', '9', '123456789', '123456789', '7', '123456789', '123456789', '123456789', '123456789', '123456789', '123456789', '123456789', '8', '123456789', '123456789', '6', '7', '123456789', '8', '2', '123456789', '123456789', '123456789', '123456789', '2', '6', '123456789', '9', '5', '123456789', '123456789', '8', '123456789', '123456789', '2', '123456789', '3', '123456789', '123456789', '9', '123456789', '123456789', '5', '123456789', '1', '123456789', '3', '123456789', '123456789']


In [31]:
display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

# eliminate()

In [56]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    for box,value in values.items():
        if len(value) == 1:
            # Remove now the value from all the peers. 
            # So the first thing to do is to loop over all the peers,
            # being rows, cols or units
            for peer in peers[box]:
                values[peer] = values[peer].replace(value, "")
    return values

In [127]:
my_initial_sudoku = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')

In [128]:
from copy import deepcopy

In [129]:
my_initial_sudoku
my_second_sudoku = deepcopy(my_initial_sudoku)
my_second_sudoku = eliminate(my_second_sudoku)
display(my_second_sudoku)

   45    4578    3   |   9      2      14  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |   59   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6      8      9   |   5     1478    47  
   8     1467    47  |   2      5      3   |   17    1467    9   
   6      69     5   |   4      1      7   |   3     268     26  


Simply calling the `eliminate()` function 4 times we can solve this Sudoku. This was an easy one where elimination was sufficient. In other words we had enough information to solve it without doing anything else. Is not that easy for all.

In [60]:
display(eliminate(eliminate(eliminate(eliminate(my_second_sudoku)))))

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


# only_choice()

In [150]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    # TODO: Implement only choice strategy here
    for box,value in values.items():
        
        # Now let's iterate over all digits 
        # and see if it appears only once
        if len(value) > 1: # We only check when the number of digits is > 1
            for c in value:
                for peer_list in units[box]:
                    sm = 0
                    #print(peer_list)
                    #print('>',ll)
                    for bx in peer_list:
                        #print(bx)
                        if c in values[bx]:
                            sm += 1
                    if (sm == 1):
                        values[box] = values[box].replace(value, c)
                
    return values


In [161]:
unitlist

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'],
 ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'],
 ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'],
 ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'],
 ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'],
 ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'],
 ['A9', 'B9', 'C9', 'D9', 'E9',

In [134]:
display(my_initial_sudoku)

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

In [146]:
tmp = deepcopy(my_second_sudoku)
display(my_second_sudoku)


   45    4578    3   |   9      2      14  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |   59   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6      8      9   |   5     1478    47  
   8     1467    47  |   2      5      3   |   17    1467    9   
   6      69     5   |   4      1      7   |   3     268     26  


In [154]:
tmp2 = deepcopy(tmp)
display(only_choice(tmp2))

  45    8     3   |  9     2     1   |  6    5789   57  
  9     6     7   |  3     4     5   |  8     2     1   
  2     5     1   |  8     7     6   |  4     9     3   
------------------+------------------+------------------
 345   345    8   |  1    3456   2   |  9   34567   6   
  7     2     9   |  5   34569   4   |  1   13456   8   
 1345 13459   6   |  7    3459   8   |  2    1345  345  
------------------+------------------+------------------
 134   1347   2   |  6     8     9   |  5    1478   47  
  8    1467   4   |  2     5     3   |  7     6     9   
  6     9     5   |  4     1     7   |  3     8     2   


# reduce_puzzle()

In [194]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        eliminate(values)
        # Your code here: Use the Only Choice Strategy
        only_choice(values)
        
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values


In [204]:
tmp = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
tmp2 = reduce_puzzle(tmp)
display(reduce_puzzle(tmp2))

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


# Other tests

In [182]:
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid2)
display(reduce_puzzle(values))

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


In [183]:
tmp2 = reduce_puzzle(values)
len([box for box in tmp2.keys() if len(tmp2[box]) == 0])

0

In [184]:
display(tmp2)

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


The method did not solve this more difficult puzzle... Some improvements are needed

# Most difficult Sudoku (http://www.conceptispuzzles.com/de/index.aspx?uri=info/article/424)

In [199]:
sud = grid_values('8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..')
display(sud)

    8     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789     3     |    6     123456789 123456789 |123456789 123456789 123456789 
123456789     7     123456789 |123456789     9     123456789 |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     5     123456789 |123456789 123456789     7     |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789     4         5     |    7     123456789 123456789 
123456789 123456789 123456789 |    1     123456789 123456789 |123456789     3     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     1     |123456789 123456789 123456789 |123456789     6         8     
123456789 123456789     8     |    5     123456789 123456789 |123456789     1     123456789 
123456789     9     123456789 |123456789 123456789 123456789 |    4   

In [207]:
display(grid_values_empty('8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..'))

8 . . |. . . |. . . 
. . 3 |6 . . |. . . 
. 7 . |. 9 . |2 . . 
------+------+------
. 5 . |. . 7 |. . . 
. . . |. 4 5 |7 . . 
. . . |1 . . |. 3 . 
------+------+------
. . 1 |. . . |. 6 8 
. . 8 |5 . . |. 1 . 
. 9 . |. . . |4 . . 


In [200]:
display(reduce_puzzle(sud))

   8      1246   24569  |  2347   12357    1234  | 13569    4579  1345679 
 12459    124      3    |   6     12578    1248  |  1589   45789   14579  
  1456     7      456   |  348      9      1348  |   2      458    13456  
------------------------+------------------------+------------------------
 123469    5      2469  |  2389    2368     7    |  1689    2489   12469  
 12369   12368    269   |  2389     4       5    |   7      289     1269  
 24679    2468   24679  |   1      268     2689  |  5689     3     24569  
------------------------+------------------------+------------------------
 23457    234      1    | 23479    237     2349  |  359      6       8    
 23467    2346     8    |   5      2367   23469  |   39      1      2379  
 23567     9      2567  |  2378   123678  12368  |   4      257     2357  


# search()

In [202]:
def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    
    # Now use recursion to solve each one of the resulting sudokus, and if one returns a value (not False), 
    # return that answer!

    # If you're stuck, see the solution.py tab!
    
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [203]:
display(search(sud))

8 1 2 |7 5 3 |6 4 9 
9 4 3 |6 8 2 |1 7 5 
6 7 5 |4 9 1 |2 8 3 
------+------+------
1 5 4 |2 3 7 |8 9 6 
3 6 9 |8 4 5 |7 2 1 
2 8 7 |1 6 9 |5 3 4 
------+------+------
5 2 1 |9 7 4 |3 6 8 
4 3 8 |5 2 6 |9 1 7 
7 9 6 |3 1 8 |4 5 2 
